https://minerandodados.com.br/como-salvar-um-modelo-de-machine-learning-em-disco/

# Adoption System

PetFinder.my Adoption Prediction <br>
Dataset Souce: https://www.kaggle.com/c/petfinder-adoption-prediction/data

## Setup

### Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
import joblib
from sklearn.model_selection import train_test_split

In [2]:
#Display all columns
pd.set_option('display.max_columns', None)

### Import Dataset

In [5]:
#Import train dataset
df_model = pd.read_csv('../final_dataset/train.csv')

#Import test dataset
df_ong = pd.read_csv('../final_dataset/test.csv')

#Import Breed Dictionary
breed_labels = pd.read_csv('../BreedLabels.csv')

#Import Colors Dictionary
color_labels = pd.read_csv('../ColorLabels.csv')

#Import State Dictionary
state_labels = pd.read_csv('../StateLabels.csv')


#Display all columns
pd.set_option('display.max_columns', None)

In [6]:
#Check the first lines
df_model.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,AdoptionSpeed,NonAdopted,Fee_y
0,2,Nibble,3,299,0,1,1,7,0,1,1,2,2,2,1,1,100,41326,8480853f516546f6cf33aa88cd76c379,0,Nibble is a 3+ month old ball of cuteness. He ...,86e1089a3,1.0,2,0,1
1,2,No Name Yet,1,265,0,1,1,2,0,2,2,3,3,3,1,1,0,41401,3082c7125d8fb66f7dd4bff4192c8b14,0,I just found it alone yesterday near my apartm...,6296e909a,2.0,0,0,0
2,1,Brisco,1,307,0,1,2,7,0,2,2,1,1,2,1,1,0,41326,fa90fa5b1ee11c86938398b60abc32cb,0,Their pregnant mother was dumped by her irresp...,3422e4906,7.0,3,0,0
3,1,Miko,4,307,0,2,1,2,0,2,1,1,1,2,1,1,150,41401,9238e4f44c71a75282e62f7136c6b240,0,"Good guard dog, very alert, active, obedience ...",5842f1ff5,8.0,2,0,1
4,1,Hunter,1,307,0,1,1,0,0,2,1,2,2,2,1,1,0,41326,95481e953f8aed9ec3d16fc4509537e8,0,This handsome yet cute boy is up for adoption....,850a43f90,3.0,2,0,0


In [7]:
#Check the first lines
df_ong.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,Fee_y,Photo
0,2,Chi Chi,36,285,264,2,1,4,7,2,3,1,1,1,2,1,0,41326,2ece3b2573dcdcebd774e635dca15fd9,0,"Please note that Chichi has been neutered, the...",f153b465f,1.0,0,f153b465f-1
1,2,Sticky,2,265,0,1,6,7,0,2,2,1,1,2,1,1,200,41326,e59c106e9912fa30c898976278c2e834,0,"Sticky, named such because of his tendency to ...",3c90f3f54,4.0,1,3c90f3f54-4
2,2,Cuddles,12,265,0,1,2,3,7,2,2,1,1,1,1,1,0,41326,e59c106e9912fa30c898976278c2e834,0,"Extremely cuddly cat, hence the origin of his ...",09f0df7d1,5.0,0,09f0df7d1-5
3,2,Porridge,4,265,0,1,6,7,0,2,2,1,1,2,1,1,0,41326,e59c106e9912fa30c898976278c2e834,0,Porridge the cat is looking for a place to cal...,0487529d4,3.0,0,0487529d4-3
4,2,Sultan [In Shah Alam],24,265,0,1,6,7,0,2,2,1,1,1,1,1,0,41326,e59c106e9912fa30c898976278c2e834,0,We first met Sultan on our campus hiding from ...,bae7c4b1c,2.0,0,bae7c4b1c-2


##  Pipeline steps:

`preprocessing` $\to$ `modelling`

### Preprocessing

#### Categorical columns

In [23]:
cat_columns = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
               'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
               'Sterilized', 'Health', 'State', 'Fee_y']

##### Creating categorical encoder 

In [24]:
encoder = TargetEncoder(smoothing=0,cols=cat_columns)

##### Standardization

In [25]:
scaler = StandardScaler()

##### Create a pipeline for categorical columns

In [26]:
categorical_pipeline = Pipeline(steps=[('encoder', encoder),
                                       ('standardize', scaler)])

#### Numerical columns

In [27]:
num_columns = ['Age']
num_columns

['Age']

##### Create a pipeline for  numerical columns

In [28]:
from sklearn.impute import SimpleImputer, KNNImputer
numerical_pipeline = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean'))])

### Create Pipeline 

In [29]:
from sklearn.compose import ColumnTransformer

preprocessing = ColumnTransformer(transformers=[('numerical_transform', numerical_pipeline, num_columns),
                                                ('categ_transform', categorical_pipeline, cat_columns)])

In [30]:
pipeline = Pipeline(steps=[('preprocessing', preprocessing),
                           ('scaler',StandardScaler()),
                           ('modelling', LogisticRegression(penalty='l1', C=0.01, solver='saga'))])

### Pickle

In [88]:
joblib.dump(pipeline,'../model-v2.pkl')

['../model-v2.pkl']

In [ ]:
#pipeline = joblib.load('model-v1.pkl')
#pipeline

### Applying pipeline - Model Dataset

In [31]:
df_model.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed',
       'NonAdopted', 'Fee_y'],
      dtype='object')

In [32]:
X_model = df_model[['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
               'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
               'Sterilized', 'Health', 'State', 'Fee_y']]

y_model = df_model['NonAdopted']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, stratify=y_model)

In [34]:
pipeline.fit(X_train, y_train)

c:\users\letic\appdata\local\programs\python\python38\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('numerical_transform',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Age']),
                                                 ('categ_transform',
                                                  Pipeline(steps=[('encoder',
                                                                   TargetEncoder(cols=['Type',
                                                                                       'Breed1',
                                                                                       'Breed2',
                                                                                       'Gender',
                                                                                       'Color1',
                                      

#### Coeficients

In [35]:
pipeline.named_steps['modelling'].coef_

array([[ 0.16744243, -0.01172201,  0.46766953,  0.15193124,  0.        ,
         0.01799651,  0.02232874,  0.        ,  0.        ,  0.0189402 ,
         0.02119891,  0.        ,  0.3850451 ,  0.03760772,  0.16660066,
         0.05464001]])

In [39]:
# Checking the coefficients
pd.DataFrame(zip(X_model.columns, pipeline.named_steps['modelling'].coef_.tolist()[0]), 
             columns=['feature', 'coefficient']).sort_values(by='coefficient', ascending=False).reset_index(drop=True)

,feature,coefficient
0,Breed1,0.467670
1,Sterilized,0.385045
2,Type,0.167442
3,State,0.166601
4,Breed2,0.151931
5,Fee_y,0.054640
6,Health,0.037608
7,Color2,0.022329
8,Vaccinated,0.021199
9,FurLength,0.018940


**Insights:**<br>
Breed1 - 0.467670 <br>
Sterilized - 0.385045 <br>
Type - 0.167442 <br>

#### Probability Result

In [49]:
pipeline.predict_proba(X_test)

array([[0.90822839, 0.09177161],
       [0.89696296, 0.10303704],
       [0.74242589, 0.25757411],
       ...,
       [0.85183017, 0.14816983],
       [0.68104281, 0.31895719],
       [0.9080837 , 0.0919163 ]])

#### Roc Curve

In [50]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1]))
print(roc_auc_score(y_train, pipeline.predict_proba(X_train)[:, 1]))

0.6822579164473603
0.7130687218381702


## Results

### Applying pipeline - ONG Dataset

In [51]:
df_ong.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'Fee_y', 'Photo',
       'P_non_adopt'],
      dtype='object')

In [52]:
X_ong = df_ong[['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
               'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
               'Sterilized', 'Health', 'Quantity', 'State', 'Fee_y']]

##### Calculate probability of NonAdopted - ONG Dataset

In [53]:
#Probabilidade dos pets não serem adotados
P_non_adopt = pipeline.predict_proba(X_ong)[:,1]
P_non_adopt

c:\users\letic\appdata\local\programs\python\python38\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "


array([0.85981024, 0.85955557, 0.80169251, ..., 0.03581526, 0.03503586,
       0.03097445])

##### Create a column with probability on ONG Dataset

In [54]:
df_ong['P_non_adopt'] = P_non_adopt
df_ong.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,Fee_y,Photo,P_non_adopt
1244,2,Shuti,120,293,0,2,2,5,7,2,2,1,1,1,1,1,0,41401,c4e8a83ec42a74cdaf4c500e0b3ee686,0,Kitty is a beautiful cat. Owner moving away an...,30e0fa4e6,3.0,0,30e0fa4e6-3,0.859810
1823,2,Kak Nyeng,4,284,283,2,4,0,0,3,2,1,1,3,1,1,0,41326,c0934d2417dd150bf7dc17e194106e05,0,Hai. Saya ada seekor kucing betina utk adopt c...,1d1ce8c48,3.0,0,1d1ce8c48-3,0.859556
1728,2,Chimmy,17,250,0,1,1,0,0,1,1,1,1,1,1,1,0,41401,4c1e498c466a0561d0daa591ffa06293,0,I picked him up at 3 months old and he has liv...,9d99bd0c3,5.0,0,9d99bd0c3-5,0.801693
2952,2,Adolf,24,296,0,1,2,5,7,2,1,2,2,2,1,1,0,41326,34b95aac1f0546a9e7101a9723b51b4c,0,I found him in cyberjaya. I believe he used to...,b05179874,3.0,0,b05179874-3,0.798342
263,1,Norok,108,202,0,2,1,2,0,2,2,1,1,1,1,1,0,41327,8019146babc11f5fd87ad8b7d530f374,0,The owner is moving back to states and is worr...,6082976d9,0.0,0,6082976d9-0,0.778115
10,1,Edison ****,17,173,0,1,2,7,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,Malacca Anyone lost their dog around Stadium H...,c2bbbdde2,2.0,0,c2bbbdde2-2,0.770460
14,1,Desmond ****,60,307,0,1,1,4,0,2,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Desmond, male, around 5 – 6 years old....",cee2cdc6b,4.0,0,cee2cdc6b-4,0.768864
12,1,Hugo ****,5,173,0,1,1,7,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Pit Bull puppy looking for home! Male,...",9e11f1974,2.0,0,9e11f1974-2,0.748820
2710,1,Cheras,24,307,307,1,2,3,0,3,2,3,3,3,3,1,1000,41324,9ad1b958576d0cba95106bc8ba130227,0,We have just rescued a dog at 18.5km grand sag...,d49c7fc09,1.0,1,d49c7fc09-1,0.748306
1831,2,Kitty,3,250,0,2,1,0,0,1,2,2,2,1,1,1,0,41326,c0aa0f250b3f2416dafb891ffc905ec5,0,"She's currently feeding on cat biscuits, need ...",171e38472,2.0,0,171e38472-2,0.734908


#### Separete into 02 datasets:

##### If dog option it's selected:

In [70]:
df_ong_dog = df_ong.loc[df_ong['Type']==1]
df_ong_dog.sort_values(['P_non_adopt'], ascending = False, inplace = True)
df_ong_dog.head(10)

<ipython-input-70-85b14bbc7954>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ong_dog.sort_values(['P_non_adopt'], ascending = False, inplace = True)


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,Fee_y,Photo,P_non_adopt
263,1,Norok,108,202,0,2,1,2,0,2,2,1,1,1,1,1,0,41327,8019146babc11f5fd87ad8b7d530f374,0,The owner is moving back to states and is worr...,6082976d9,0.0,0,6082976d9-0,0.778115
10,1,Edison ****,17,173,0,1,2,7,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,Malacca Anyone lost their dog around Stadium H...,c2bbbdde2,2.0,0,c2bbbdde2-2,0.770460
14,1,Desmond ****,60,307,0,1,1,4,0,2,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Desmond, male, around 5 – 6 years old....",cee2cdc6b,4.0,0,cee2cdc6b-4,0.768864
12,1,Hugo ****,5,173,0,1,1,7,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Pit Bull puppy looking for home! Male,...",9e11f1974,2.0,0,9e11f1974-2,0.748820
2710,1,Cheras,24,307,307,1,2,3,0,3,2,3,3,3,3,1,1000,41324,9ad1b958576d0cba95106bc8ba130227,0,We have just rescued a dog at 18.5km grand sag...,d49c7fc09,1.0,1,d49c7fc09-1,0.748306
477,1,Ashley,96,173,0,2,1,7,0,2,1,1,1,3,1,1,0,41401,64584500a7ecf68b731b4710a95097b7,0,We found Ashley at Sri Gombak on the 12th Feb....,c873e4764,2.0,0,c873e4764-2,0.727882
16,1,Wong Loi æºä¾ ****,36,307,0,1,2,7,0,2,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Anyone lost their dog? Male, around 3 ...",34af29aab,3.0,0,34af29aab-3,0.722422
20,1,BoBo ****,36,307,85,2,1,2,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca BoBo, Female, around 3 to 4 years old....",6b593bbe2,5.0,0,6b593bbe2-5,0.712263
222,1,No Name,48,76,0,1,1,7,0,3,1,3,3,3,1,1,0,41327,8019146babc11f5fd87ad8b7d530f374,0,This dog was found at the junction of YMCA and...,1132dae5c,7.0,0,1132dae5c-7,0.688399
1738,1,Machiato,3,307,0,2,1,0,0,2,1,1,1,1,1,1,0,41324,e243ffc523755488c9f3a24926c7a087,0,"Short tail BB, active, cute and adorable. Plea...",603b290d1,3.0,0,603b290d1-3,0.678763


##### If cat option it's selected:

In [69]:
df_ong_cat = df_ong[df_ong['Type']==2]
df_ong_cat.sort_values(['P_non_adopt'], ascending = False, inplace = True)
df_ong_cat.head(10)

<ipython-input-69-d3d07cad85c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ong_cat.sort_values(['P_non_adopt'], ascending = False, inplace = True)


,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,Fee_y,Photo,P_non_adopt
1244,2,Shuti,120,293,0,2,2,5,7,2,2,1,1,1,1,1,0,41401,c4e8a83ec42a74cdaf4c500e0b3ee686,0,Kitty is a beautiful cat. Owner moving away an...,30e0fa4e6,3.0,0,30e0fa4e6-3,0.859810
1823,2,Kak Nyeng,4,284,283,2,4,0,0,3,2,1,1,3,1,1,0,41326,c0934d2417dd150bf7dc17e194106e05,0,Hai. Saya ada seekor kucing betina utk adopt c...,1d1ce8c48,3.0,0,1d1ce8c48-3,0.859556
1728,2,Chimmy,17,250,0,1,1,0,0,1,1,1,1,1,1,1,0,41401,4c1e498c466a0561d0daa591ffa06293,0,I picked him up at 3 months old and he has liv...,9d99bd0c3,5.0,0,9d99bd0c3-5,0.801693
2952,2,Adolf,24,296,0,1,2,5,7,2,1,2,2,2,1,1,0,41326,34b95aac1f0546a9e7101a9723b51b4c,0,I found him in cyberjaya. I believe he used to...,b05179874,3.0,0,b05179874-3,0.798342
1831,2,Kitty,3,250,0,2,1,0,0,1,2,2,2,1,1,1,0,41326,c0aa0f250b3f2416dafb891ffc905ec5,0,"She's currently feeding on cat biscuits, need ...",171e38472,2.0,0,171e38472-2,0.734908
516,2,Tabby,12,284,0,2,6,7,0,2,1,1,1,1,1,1,0,41401,64584500a7ecf68b731b4710a95097b7,0,"Tabby was rescued a month ago by my colleague,...",a590480ae,1.0,0,a590480ae-1,0.704699
2752,2,White Boy,36,283,0,1,7,0,0,2,2,1,1,1,1,1,0,41330,eed6fda4263e862878ebf67d02e483f0,0,"A very ""manja"" 3 years old healthy male cat up...",eefa95b87,3.0,0,eefa95b87-3,0.675530
1457,2,Meshki,12,245,0,1,1,0,0,2,2,1,1,1,1,1,0,41326,5aba8b86a7424474dae183ac383d7548,0,He is a calm and friendly cat. I am not sure a...,f3a641543,2.0,0,f3a641543-2,0.675199
1612,2,Max,5,243,268,1,1,0,0,2,1,1,1,1,1,1,100,41401,fe3a548d3037407161ec79c4d6b7fbbc,0,Muscular built with the sweetest character.,8593527d2,2.0,1,8593527d2-2,0.659962
2848,2,Cicak,2,283,283,2,6,0,0,2,2,2,2,2,1,1,0,41401,c3c566ca3501e1827b410caed82dc0ae,0,i've to move in a new condominium.sadly i cant...,1e39eb7fa,1.0,0,1e39eb7fa-1,0.640320


##### Sort the first 10 - geral

In [46]:
df_ong.sort_values(['P_non_adopt'], ascending = False, inplace = True)
df_ong.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,Fee_y,Photo,P_non_adopt
1244,2,Shuti,120,293,0,2,2,5,7,2,2,1,1,1,1,1,0,41401,c4e8a83ec42a74cdaf4c500e0b3ee686,0,Kitty is a beautiful cat. Owner moving away an...,30e0fa4e6,3.0,0,30e0fa4e6-3,0.859810
1823,2,Kak Nyeng,4,284,283,2,4,0,0,3,2,1,1,3,1,1,0,41326,c0934d2417dd150bf7dc17e194106e05,0,Hai. Saya ada seekor kucing betina utk adopt c...,1d1ce8c48,3.0,0,1d1ce8c48-3,0.859556
1728,2,Chimmy,17,250,0,1,1,0,0,1,1,1,1,1,1,1,0,41401,4c1e498c466a0561d0daa591ffa06293,0,I picked him up at 3 months old and he has liv...,9d99bd0c3,5.0,0,9d99bd0c3-5,0.801693
2952,2,Adolf,24,296,0,1,2,5,7,2,1,2,2,2,1,1,0,41326,34b95aac1f0546a9e7101a9723b51b4c,0,I found him in cyberjaya. I believe he used to...,b05179874,3.0,0,b05179874-3,0.798342
263,1,Norok,108,202,0,2,1,2,0,2,2,1,1,1,1,1,0,41327,8019146babc11f5fd87ad8b7d530f374,0,The owner is moving back to states and is worr...,6082976d9,0.0,0,6082976d9-0,0.778115
10,1,Edison ****,17,173,0,1,2,7,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,Malacca Anyone lost their dog around Stadium H...,c2bbbdde2,2.0,0,c2bbbdde2-2,0.770460
14,1,Desmond ****,60,307,0,1,1,4,0,2,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Desmond, male, around 5 – 6 years old....",cee2cdc6b,4.0,0,cee2cdc6b-4,0.768864
12,1,Hugo ****,5,173,0,1,1,7,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Pit Bull puppy looking for home! Male,...",9e11f1974,2.0,0,9e11f1974-2,0.748820
2710,1,Cheras,24,307,307,1,2,3,0,3,2,3,3,3,3,1,1000,41324,9ad1b958576d0cba95106bc8ba130227,0,We have just rescued a dog at 18.5km grand sag...,d49c7fc09,1.0,1,d49c7fc09-1,0.748306
1831,2,Kitty,3,250,0,2,1,0,0,1,2,2,2,1,1,1,0,41326,c0aa0f250b3f2416dafb891ffc905ec5,0,"She's currently feeding on cat biscuits, need ...",171e38472,2.0,0,171e38472-2,0.734908


### Replacing categotical columns

#### State

In [47]:
my_dict = {state_labels.iloc[i,0]: state_labels.iloc[i,1] for i in range(len(state_labels['StateName']))}
my_dict

{41336: 'Johor',
 41325: 'Kedah',
 41367: 'Kelantan',
 41401: 'Kuala Lumpur',
 41415: 'Labuan',
 41324: 'Melaka',
 41332: 'Negeri Sembilan',
 41335: 'Pahang',
 41330: 'Perak',
 41380: 'Perlis',
 41327: 'Pulau Pinang',
 41345: 'Sabah',
 41342: 'Sarawak',
 41326: 'Selangor',
 41361: 'Terengganu'}

#### Breed1 & Breed2

In [48]:
my_dict_breed = {breed_labels.iloc[i,0]: breed_labels.iloc[i,2] for i in range(len(breed_labels['BreedName']))}
my_dict_breed 

{1: 'Affenpinscher',
 2: 'Afghan Hound',
 3: 'Airedale Terrier',
 4: 'Akbash',
 5: 'Akita',
 6: 'Alaskan Malamute',
 7: 'American Bulldog',
 8: 'American Eskimo Dog',
 9: 'American Hairless Terrier',
 10: 'American Staffordshire Terrier',
 11: 'American Water Spaniel',
 12: 'Anatolian Shepherd',
 13: 'Appenzell Mountain Dog',
 14: 'Australian Cattle Dog/Blue Heeler',
 15: 'Australian Kelpie',
 16: 'Australian Shepherd',
 17: 'Australian Terrier',
 18: 'Basenji',
 19: 'Basset Hound',
 20: 'Beagle',
 21: 'Bearded Collie',
 22: 'Beauceron',
 23: 'Bedlington Terrier',
 24: 'Belgian Shepherd Dog Sheepdog',
 25: 'Belgian Shepherd Laekenois',
 26: 'Belgian Shepherd Malinois',
 27: 'Belgian Shepherd Tervuren',
 28: 'Bernese Mountain Dog',
 29: 'Bichon Frise',
 30: 'Black and Tan Coonhound',
 31: 'Black Labrador Retriever',
 32: 'Black Mouth Cur',
 33: 'Black Russian Terrier',
 34: 'Bloodhound',
 35: 'Blue Lacy',
 36: 'Bluetick Coonhound',
 37: 'Boerboel',
 38: 'Bolognese',
 39: 'Border Collie'

#### Gender

In [73]:
my_dict_gender = {1: 'Male', 2: 'Female', 3: 'Mixed'}

#### Color1, Color2 & Color3

In [74]:
my_dict_color = {color_labels.iloc[i,0]: color_labels.iloc[i,1] for i in range(len(color_labels['ColorName']))}

####  MaturitySize

In [75]:
my_dict_maturity = {1: 'Small', 2: 'Medium', 3: 'Large', 4: 'Extra Large', 0: 'Not Specified'}

####  FurLength

In [76]:
my_dict_length = {1: 'Short', 2: 'Medium', 3: 'Long', 0: 'Not Specified'}

####  Vaccinated 

In [77]:
my_dict_vaccinated = {1: 'Yes', 2: 'No', 3: 'Not Sure'}

#### Dewormed

In [78]:
my_dict_dewormed = {1:'Yes', 2:'No', 3:'Not Sure'}

#### Sterilized 

In [79]:
my_dict_sterilized = {1:'Yes', 2:'No', 3:'Not Sure'}

#### Health

In [80]:
my_dict_health = {1:'Healthy', 2:'Minor Injury', 3:'Serious Injury', 0:'Not Specified'}

#### Replace All

##### All

In [81]:
df_ong = df_ong.replace({'Breed1':my_dict_breed ,'Breed2':my_dict_breed, 'Gender':my_dict_gender, 'Color1':my_dict_color, 
                'Color2':my_dict_color,'Color3':my_dict_color,'State':my_dict, 'MaturitySize':my_dict_maturity,
               'FurLength':my_dict_length, 'Vaccinated':my_dict_vaccinated, 'Dewormed':my_dict_dewormed,
               'Sterilized':my_dict_sterilized, 'Health':my_dict_health})

##### Cat

In [82]:
df_ong_cat = df_ong_cat.replace({'Breed1':my_dict_breed ,'Breed2':my_dict_breed, 'Gender':my_dict_gender, 'Color1':my_dict_color, 
                'Color2':my_dict_color,'Color3':my_dict_color,'State':my_dict, 'MaturitySize':my_dict_maturity,
               'FurLength':my_dict_length, 'Vaccinated':my_dict_vaccinated, 'Dewormed':my_dict_dewormed,
               'Sterilized':my_dict_sterilized, 'Health':my_dict_health})

##### Dog

In [ ]:
df_ong_dog = df_ong_dog.replace({'Breed1':my_dict_breed ,'Breed2':my_dict_breed, 'Gender':my_dict_gender, 'Color1':my_dict_color, 
                'Color2':my_dict_color,'Color3':my_dict_color,'State':my_dict, 'MaturitySize':my_dict_maturity,
               'FurLength':my_dict_length, 'Vaccinated':my_dict_vaccinated, 'Dewormed':my_dict_dewormed,
               'Sterilized':my_dict_sterilized, 'Health':my_dict_health})

In [84]:
df_ong.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,Fee_y,Photo,P_non_adopt
1244,2,Shuti,120,Siberian,0,Female,Brown,Cream,White,Medium,Medium,Yes,Yes,Yes,Healthy,1,0,Kuala Lumpur,c4e8a83ec42a74cdaf4c500e0b3ee686,0,Kitty is a beautiful cat. Owner moving away an...,30e0fa4e6,3.0,0,30e0fa4e6-3,0.859810
1823,2,Kak Nyeng,4,Oriental Tabby,Oriental Short Hair,Female,Yellow,0,0,Large,Medium,Yes,Yes,Not Sure,Healthy,1,0,Selangor,c0934d2417dd150bf7dc17e194106e05,0,Hai. Saya ada seekor kucing betina utk adopt c...,1d1ce8c48,3.0,0,1d1ce8c48-3,0.859556
1728,2,Chimmy,17,Bombay,0,Male,Black,0,0,Small,Short,Yes,Yes,Yes,Healthy,1,0,Kuala Lumpur,4c1e498c466a0561d0daa591ffa06293,0,I picked him up at 3 months old and he has liv...,9d99bd0c3,5.0,0,9d99bd0c3-5,0.801693
2952,2,Adolf,24,Snowshoe,0,Male,Brown,Cream,White,Medium,Short,No,No,No,Healthy,1,0,Selangor,34b95aac1f0546a9e7101a9723b51b4c,0,I found him in cyberjaya. I believe he used to...,b05179874,3.0,0,b05179874-3,0.798342
263,1,Norok,108,Shepherd,0,Female,Black,Brown,0,Medium,Medium,Yes,Yes,Yes,Healthy,1,0,Pulau Pinang,8019146babc11f5fd87ad8b7d530f374,0,The owner is moving back to states and is worr...,6082976d9,0.0,0,6082976d9-0,0.778115
10,1,Edison ****,17,Pit Bull Terrier,0,Male,Brown,White,0,Large,Short,Not Sure,Not Sure,Not Sure,Healthy,1,0,Melaka,cd5cb3f83ab1387406a177c5bbf1477a,0,Malacca Anyone lost their dog around Stadium H...,c2bbbdde2,2.0,0,c2bbbdde2-2,0.770460
14,1,Desmond ****,60,Mixed Breed,0,Male,Black,Yellow,0,Medium,Short,Not Sure,Not Sure,Not Sure,Healthy,1,0,Melaka,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Desmond, male, around 5 – 6 years old....",cee2cdc6b,4.0,0,cee2cdc6b-4,0.768864
12,1,Hugo ****,5,Pit Bull Terrier,0,Male,Black,White,0,Large,Short,Not Sure,Not Sure,Not Sure,Healthy,1,0,Melaka,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Pit Bull puppy looking for home! Male,...",9e11f1974,2.0,0,9e11f1974-2,0.748820
2710,1,Cheras,24,Mixed Breed,Mixed Breed,Male,Brown,Golden,0,Large,Medium,Not Sure,Not Sure,Not Sure,Serious Injury,1,1000,Melaka,9ad1b958576d0cba95106bc8ba130227,0,We have just rescued a dog at 18.5km grand sag...,d49c7fc09,1.0,1,d49c7fc09-1,0.748306
1831,2,Kitty,3,Bombay,0,Female,Black,0,0,Small,Medium,No,No,Yes,Healthy,1,0,Selangor,c0aa0f250b3f2416dafb891ffc905ec5,0,"She's currently feeding on cat biscuits, need ...",171e38472,2.0,0,171e38472-2,0.734908


### Final Result - Exibited on streamlit

In [85]:
df_ong.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,Fee_y,Photo,P_non_adopt
1244,2,Shuti,120,Siberian,0,Female,Brown,Cream,White,Medium,Medium,Yes,Yes,Yes,Healthy,1,0,Kuala Lumpur,c4e8a83ec42a74cdaf4c500e0b3ee686,0,Kitty is a beautiful cat. Owner moving away an...,30e0fa4e6,3.0,0,30e0fa4e6-3,0.859810
1823,2,Kak Nyeng,4,Oriental Tabby,Oriental Short Hair,Female,Yellow,0,0,Large,Medium,Yes,Yes,Not Sure,Healthy,1,0,Selangor,c0934d2417dd150bf7dc17e194106e05,0,Hai. Saya ada seekor kucing betina utk adopt c...,1d1ce8c48,3.0,0,1d1ce8c48-3,0.859556
1728,2,Chimmy,17,Bombay,0,Male,Black,0,0,Small,Short,Yes,Yes,Yes,Healthy,1,0,Kuala Lumpur,4c1e498c466a0561d0daa591ffa06293,0,I picked him up at 3 months old and he has liv...,9d99bd0c3,5.0,0,9d99bd0c3-5,0.801693
2952,2,Adolf,24,Snowshoe,0,Male,Brown,Cream,White,Medium,Short,No,No,No,Healthy,1,0,Selangor,34b95aac1f0546a9e7101a9723b51b4c,0,I found him in cyberjaya. I believe he used to...,b05179874,3.0,0,b05179874-3,0.798342
263,1,Norok,108,Shepherd,0,Female,Black,Brown,0,Medium,Medium,Yes,Yes,Yes,Healthy,1,0,Pulau Pinang,8019146babc11f5fd87ad8b7d530f374,0,The owner is moving back to states and is worr...,6082976d9,0.0,0,6082976d9-0,0.778115
10,1,Edison ****,17,Pit Bull Terrier,0,Male,Brown,White,0,Large,Short,Not Sure,Not Sure,Not Sure,Healthy,1,0,Melaka,cd5cb3f83ab1387406a177c5bbf1477a,0,Malacca Anyone lost their dog around Stadium H...,c2bbbdde2,2.0,0,c2bbbdde2-2,0.770460
14,1,Desmond ****,60,Mixed Breed,0,Male,Black,Yellow,0,Medium,Short,Not Sure,Not Sure,Not Sure,Healthy,1,0,Melaka,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Desmond, male, around 5 – 6 years old....",cee2cdc6b,4.0,0,cee2cdc6b-4,0.768864
12,1,Hugo ****,5,Pit Bull Terrier,0,Male,Black,White,0,Large,Short,Not Sure,Not Sure,Not Sure,Healthy,1,0,Melaka,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Pit Bull puppy looking for home! Male,...",9e11f1974,2.0,0,9e11f1974-2,0.748820
2710,1,Cheras,24,Mixed Breed,Mixed Breed,Male,Brown,Golden,0,Large,Medium,Not Sure,Not Sure,Not Sure,Serious Injury,1,1000,Melaka,9ad1b958576d0cba95106bc8ba130227,0,We have just rescued a dog at 18.5km grand sag...,d49c7fc09,1.0,1,d49c7fc09-1,0.748306
1831,2,Kitty,3,Bombay,0,Female,Black,0,0,Small,Medium,No,No,Yes,Healthy,1,0,Selangor,c0aa0f250b3f2416dafb891ffc905ec5,0,"She's currently feeding on cat biscuits, need ...",171e38472,2.0,0,171e38472-2,0.734908


In [86]:
df_ong_dog.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,Fee_y,Photo,P_non_adopt
263,1,Norok,108,202,0,2,1,2,0,2,2,1,1,1,1,1,0,41327,8019146babc11f5fd87ad8b7d530f374,0,The owner is moving back to states and is worr...,6082976d9,0.0,0,6082976d9-0,0.778115
10,1,Edison ****,17,173,0,1,2,7,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,Malacca Anyone lost their dog around Stadium H...,c2bbbdde2,2.0,0,c2bbbdde2-2,0.770460
14,1,Desmond ****,60,307,0,1,1,4,0,2,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Desmond, male, around 5 – 6 years old....",cee2cdc6b,4.0,0,cee2cdc6b-4,0.768864
12,1,Hugo ****,5,173,0,1,1,7,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Pit Bull puppy looking for home! Male,...",9e11f1974,2.0,0,9e11f1974-2,0.748820
2710,1,Cheras,24,307,307,1,2,3,0,3,2,3,3,3,3,1,1000,41324,9ad1b958576d0cba95106bc8ba130227,0,We have just rescued a dog at 18.5km grand sag...,d49c7fc09,1.0,1,d49c7fc09-1,0.748306
477,1,Ashley,96,173,0,2,1,7,0,2,1,1,1,3,1,1,0,41401,64584500a7ecf68b731b4710a95097b7,0,We found Ashley at Sri Gombak on the 12th Feb....,c873e4764,2.0,0,c873e4764-2,0.727882
16,1,Wong Loi æºä¾ ****,36,307,0,1,2,7,0,2,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca Anyone lost their dog? Male, around 3 ...",34af29aab,3.0,0,34af29aab-3,0.722422
20,1,BoBo ****,36,307,85,2,1,2,0,3,1,3,3,3,1,1,0,41324,cd5cb3f83ab1387406a177c5bbf1477a,0,"Malacca BoBo, Female, around 3 to 4 years old....",6b593bbe2,5.0,0,6b593bbe2-5,0.712263
222,1,No Name,48,76,0,1,1,7,0,3,1,3,3,3,1,1,0,41327,8019146babc11f5fd87ad8b7d530f374,0,This dog was found at the junction of YMCA and...,1132dae5c,7.0,0,1132dae5c-7,0.688399
1738,1,Machiato,3,307,0,2,1,0,0,2,1,1,1,1,1,1,0,41324,e243ffc523755488c9f3a24926c7a087,0,"Short tail BB, active, cute and adorable. Plea...",603b290d1,3.0,0,603b290d1-3,0.678763


In [87]:
df_ong_cat.head(10)

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,RescuerID,VideoAmt,Description,PetID,PhotoAmt,Fee_y,Photo,P_non_adopt
1244,2,Shuti,120,Siberian,0,Female,Brown,Cream,White,Medium,Medium,Yes,Yes,Yes,Healthy,1,0,Kuala Lumpur,c4e8a83ec42a74cdaf4c500e0b3ee686,0,Kitty is a beautiful cat. Owner moving away an...,30e0fa4e6,3.0,0,30e0fa4e6-3,0.859810
1823,2,Kak Nyeng,4,Oriental Tabby,Oriental Short Hair,Female,Yellow,0,0,Large,Medium,Yes,Yes,Not Sure,Healthy,1,0,Selangor,c0934d2417dd150bf7dc17e194106e05,0,Hai. Saya ada seekor kucing betina utk adopt c...,1d1ce8c48,3.0,0,1d1ce8c48-3,0.859556
1728,2,Chimmy,17,Bombay,0,Male,Black,0,0,Small,Short,Yes,Yes,Yes,Healthy,1,0,Kuala Lumpur,4c1e498c466a0561d0daa591ffa06293,0,I picked him up at 3 months old and he has liv...,9d99bd0c3,5.0,0,9d99bd0c3-5,0.801693
2952,2,Adolf,24,Snowshoe,0,Male,Brown,Cream,White,Medium,Short,No,No,No,Healthy,1,0,Selangor,34b95aac1f0546a9e7101a9723b51b4c,0,I found him in cyberjaya. I believe he used to...,b05179874,3.0,0,b05179874-3,0.798342
1831,2,Kitty,3,Bombay,0,Female,Black,0,0,Small,Medium,No,No,Yes,Healthy,1,0,Selangor,c0aa0f250b3f2416dafb891ffc905ec5,0,"She's currently feeding on cat biscuits, need ...",171e38472,2.0,0,171e38472-2,0.734908
516,2,Tabby,12,Oriental Tabby,0,Female,Gray,White,0,Medium,Short,Yes,Yes,Yes,Healthy,1,0,Kuala Lumpur,64584500a7ecf68b731b4710a95097b7,0,"Tabby was rescued a month ago by my colleague,...",a590480ae,1.0,0,a590480ae-1,0.704699
2752,2,White Boy,36,Oriental Short Hair,0,Male,White,0,0,Medium,Medium,Yes,Yes,Yes,Healthy,1,0,Perak,eed6fda4263e862878ebf67d02e483f0,0,"A very ""manja"" 3 years old healthy male cat up...",eefa95b87,3.0,0,eefa95b87-3,0.675530
1457,2,Meshki,12,Applehead Siamese,0,Male,Black,0,0,Medium,Medium,Yes,Yes,Yes,Healthy,1,0,Selangor,5aba8b86a7424474dae183ac383d7548,0,He is a calm and friendly cat. I am not sure a...,f3a641543,2.0,0,f3a641543-2,0.675199
1612,2,Max,5,American Shorthair,Exotic Shorthair,Male,Black,0,0,Medium,Short,Yes,Yes,Yes,Healthy,1,100,Kuala Lumpur,fe3a548d3037407161ec79c4d6b7fbbc,0,Muscular built with the sweetest character.,8593527d2,2.0,1,8593527d2-2,0.659962
2848,2,Cicak,2,Oriental Short Hair,Oriental Short Hair,Female,Gray,0,0,Medium,Medium,No,No,No,Healthy,1,0,Kuala Lumpur,c3c566ca3501e1827b410caed82dc0ae,0,i've to move in a new condominium.sadly i cant...,1e39eb7fa,1.0,0,1e39eb7fa-1,0.640320
